In [1]:
import discord
import random
import os
import asyncio

import pandas as pd
import requests

In [2]:
#########################

In [3]:
import pandas as pd
import glob
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
import pandas as pd
import numpy as np

def func_1(df):
    # Drop duplicates early on
    df = df.drop_duplicates()

    # Columns to replace NaNs
    df[['wides', 'noballs', 'byes', 'legbyes']] = df[['wides', 'noballs', 'byes', 'legbyes']].fillna(0)
    df[['wicket_type', 'player_dismissed']] = df[['wicket_type', 'player_dismissed']].fillna('')

    # Convert columns to string type where required
    cols_to_string = ['batting_team', 'bowling_team', 'striker', 'non_striker', 'bowler', 'wicket_type', 'player_dismissed']
    df[cols_to_string] = df[cols_to_string].astype(str)

    # Sort once for ease of cumulative calculations
    df = df.sort_values(by=['start_date', 'match_id', 'innings', 'ball']).reset_index(drop=True)

    # Calculate legality and bowler wicket conditions
    df['legality'] = (df['wides'] + df['noballs']) == 0
    df['bowler_wicket'] = df['wicket_type'].isin(['caught', 'bowled', 'lbw', 'stumped', 'caught and bowled', 'hit wicket', 'out'])

    # Flagging and categorizing runs
    df['islegal'] = df['legality'].astype(int)
    df['isDotforBatter'] = (df['runs_off_bat'] == 0).astype(int)
    df['isOne'] = (df['runs_off_bat'] == 1).astype(int)
    df['isTwo'] = (df['runs_off_bat'] == 2).astype(int)
    df['isThree'] = (df['runs_off_bat'] == 3).astype(int)
    df['isStrikeRotation'] = df['runs_off_bat'].isin([1, 2, 3, 5, 7]).astype(int)
    df['isFour'] = (df['runs_off_bat'] == 4).astype(int)
    df['isSix'] = (df['runs_off_bat'] == 6).astype(int)
    df['isBowlerWicket'] = df['bowler_wicket'].astype(int)
    df['is_striker_Out'] = (df['striker'] == df['player_dismissed']).astype(int)
    df['is_nonstriker_Out'] = (df['non_striker'] == df['player_dismissed']).astype(int)
    df['isWicket'] = (df['wicket_type'] != '').astype(int)
    df['total_runs'] = df['runs_off_bat'] + df['extras']
    
    # Convert relevant columns to numeric
    df['total_runs'] = pd.to_numeric(df['total_runs'], errors='coerce')
    df['isWicket'] = pd.to_numeric(df['isWicket'], errors='coerce')
    df['islegal'] = pd.to_numeric(df['islegal'], errors='coerce')

    # Fill any NaNs generated due to coercion with 0 (if appropriate for your logic)
    df['total_runs'].fillna(0, inplace=True)
    df['isWicket'].fillna(0, inplace=True)
    df['islegal'].fillna(0, inplace=True)

    # Cumulative sums for runs and wickets
    df['runs_scored'] = df.groupby(['match_id', 'innings'])['total_runs'].cumsum()
    df['wickets_down'] = df.groupby(['match_id', 'innings'])['isWicket'].cumsum()
    df['legal_balls_bowled'] = df.groupby(['match_id', 'innings'])['islegal'].cumsum()
    
    # Calculate run rate and target
    df['run_rate'] = df.apply(lambda x: (6 * x['runs_scored'] / x['legal_balls_bowled']) 
                              if x['legal_balls_bowled'] > 0 else 6 + x['runs_scored'], axis=1)
    

    # Calculate partnership runs with forward filling of last fall of wicket (fow) information
    df['last_fow'] = df['runs_scored'].where(df['isWicket'] == 1).ffill().fillna(0)
    df['partnership_runs'] = df['runs_scored'] - df['last_fow']

    # Calculate cumulative balls faced by each striker and non-striker
    df['isno'] = df['noballs'] > 0
    df['is_faced_by_batter'] = (df['islegal'] | df['isno']).astype(int)

    # Cumulative balls and runs for striker and non-striker
    striker_runs = []
    non_striker_runs = []
    striker_balls = []
    non_striker_balls = []
    
    
    # Calculate last fall of wicket (fow) run rate
    df['last_fow_run_rate'] = df.groupby(['match_id', 'innings'])['run_rate'].apply(lambda x: x.where(df['isWicket'] == 1).ffill())
    df.reset_index(drop=True, inplace=True)

    return df


In [5]:
def func_2(df_1):


    #### creation of fours_hit, sixes_hit

    # Sort the DataFrame by 'match_id', 'innings' for forward filling
    df_1.sort_values(by=['match_id', 'start_date', 'innings', 'ball'], inplace=True)

    # Group by 'match_id' and 'innings' and forward-fill NaN values within each group
    df_1['fours'] = df_1.groupby(['match_id', 'innings'])['isFour'].cumsum()
    df_1['sixes'] = df_1.groupby(['match_id', 'innings'])['isSix'].cumsum()

    # Reset the DataFrame index
    df_1.reset_index(drop=True, inplace=True)

    # Initialize a dictionary to store player-to-number mappings for each match_id
    player_to_number = {}

    # Initialize a list to store player_number
    player_number = [1]

    # Function to update player-to-number mappings
    def update_player_mapping(player):
        player_to_number[player] = player_number[0]
        player_number[0] += 1

    # Lists to store batting positions for striker and non-striker
    striker_positions = []
    non_striker_positions = []

    # Initialize current match_id
    current_match_id = None

    # Iterate through DataFrame rows
    for index, row in df_1.iterrows():
        #print(index)

        match_id = row['match_id']
        striker = row['striker']
        non_striker = row['non_striker']

        # Check if it's a new match_id
        if current_match_id != match_id:
            current_match_id = match_id
            player_to_number.clear()
            player_number[0] = 1

        # Check if striker has been encountered before in this match
        if striker not in player_to_number:
            update_player_mapping(striker)

        # Check if non-striker has been encountered before in this match
        if non_striker not in player_to_number:
            update_player_mapping(non_striker)

        # Append player numbers to the respective lists
        striker_positions.append(player_to_number[striker])
        non_striker_positions.append(player_to_number[non_striker])

    # Create new columns 'striker_batting_position' and 'non_striker_batting_position'
    df_1['striker_batting_position'] = striker_positions
    df_1['non_striker_batting_position'] = non_striker_positions


    return df_1

In [7]:
#ground stats
df_g = pd.read_csv('/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/ground_stats.csv')

#bowler stats
df_bo_1 = pd.read_csv('/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/bowler_stats_inn1.csv')
df_bo_2 = pd.read_csv('/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/bowler_stats_inn2.csv')

df_bo = pd.concat([df_bo_1,df_bo_2], axis=0)
df_bo.reset_index(drop=True, inplace=True)

#batter stats
df_ba_1 = pd.read_csv('/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/batter_stats_inn1.csv')
df_ba_2 = pd.read_csv('/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/batter_stats_inn2.csv')

df_ba = pd.concat([df_ba_1,df_ba_2], axis=0)
df_ba.reset_index(drop=True, inplace=True)

#h2h factor
h2h = pd.read_csv('/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/h2h_stats.csv')

#form factor-batter
form_ba_1 = pd.read_csv('/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/batter_stats_form_inn1.csv')
form_ba_2 = pd.read_csv('/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/batter_stats_form_inn2.csv')

form_ba = pd.concat([form_ba_1,form_ba_2], axis=0)
form_ba.reset_index(drop=True, inplace=True)

#form factor-bowler
form_bo_1 = pd.read_csv('/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/bowler_stats_form_inn1.csv')
form_bo_2 = pd.read_csv('/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/bowler_stats_form_inn2.csv')

form_bo = pd.concat([form_bo_1,form_bo_2], axis=0)
form_bo.reset_index(drop=True, inplace=True)

In [8]:
player_list = pd.read_excel('/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/player_list.xlsx',
                           sheet_name='players_sets')

In [9]:
def team_map(value):
    if value=='CSK':
        return team_CSK
    elif value=='DC':
        return team_DC
    elif value=='GT':
        return team_GT
    elif value=='KKR':
        return team_KKR
    elif value=='LSG':
        return team_LSG
    elif value=='MI':
        return team_MI
    elif value=='PBKS':
        return team_PBKS
    elif value=='RCB':
        return team_RCB
    elif value=='RR':
        return team_RR
    elif value=='SRH':
        return team_SRH
    else:
        return pd.DataFrame()

In [10]:
team_CSK = pd.read_excel('/Users/roumyadas/Desktop/Data/t20_cricket_data/Teams/SQUADS.xlsx', 
                        sheet_name='CSK')
team_CSK = team_CSK.sort_values(by='XI').head(11)
team_CSK = team_CSK.merge(player_list[['Name','Func_Name']], on='Name', how='left')
team_CSK['Func_Name'] = team_CSK['Func_Name'].fillna(team_CSK['Name'])

In [11]:
team_DC = pd.read_excel('/Users/roumyadas/Desktop/Data/t20_cricket_data/Teams/SQUADS.xlsx', 
                        sheet_name='DC')
team_DC = team_DC.sort_values(by='XI').head(11)
team_DC  = team_DC.merge(player_list[['Name','Func_Name']], on='Name', how='left')
team_DC['Func_Name'] = team_DC['Func_Name'].fillna(team_DC['Name'])

team_GT = pd.read_excel('/Users/roumyadas/Desktop/Data/t20_cricket_data/Teams/SQUADS.xlsx', 
                        sheet_name='GT')
team_GT = team_GT.sort_values(by='XI').head(11)
team_GT  = team_GT.merge(player_list[['Name','Func_Name']], on='Name', how='left')
team_GT['Func_Name'] = team_GT['Func_Name'].fillna(team_GT['Name'])

team_KKR = pd.read_excel('/Users/roumyadas/Desktop/Data/t20_cricket_data/Teams/SQUADS.xlsx', 
                        sheet_name='KKR')
team_KKR = team_KKR.sort_values(by='XI').head(11)
team_KKR  = team_KKR.merge(player_list[['Name','Func_Name']], on='Name', how='left')
team_KKR['Func_Name'] = team_KKR['Func_Name'].fillna(team_KKR['Name'])

team_LSG = pd.read_excel('/Users/roumyadas/Desktop/Data/t20_cricket_data/Teams/SQUADS.xlsx', 
                        sheet_name='LSG')
team_LSG = team_LSG.sort_values(by='XI').head(11)
team_LSG  = team_LSG.merge(player_list[['Name','Func_Name']], on='Name', how='left')
team_LSG['Func_Name'] = team_LSG['Func_Name'].fillna(team_LSG['Name'])

team_MI = pd.read_excel('/Users/roumyadas/Desktop/Data/t20_cricket_data/Teams/SQUADS.xlsx', 
                        sheet_name='MI')
team_MI = team_MI.sort_values(by='XI').head(11)
team_MI  = team_MI.merge(player_list[['Name','Func_Name']], on='Name', how='left')
team_MI['Func_Name'] = team_MI['Func_Name'].fillna(team_MI['Name'])

team_PBKS = pd.read_excel('/Users/roumyadas/Desktop/Data/t20_cricket_data/Teams/SQUADS.xlsx', 
                        sheet_name='PBKS')
team_PBKS = team_PBKS.sort_values(by='XI').head(11)
team_PBKS  = team_PBKS.merge(player_list[['Name','Func_Name']], on='Name', how='left')
team_PBKS['Func_Name'] = team_PBKS['Func_Name'].fillna(team_PBKS['Name'])

team_RCB = pd.read_excel('/Users/roumyadas/Desktop/Data/t20_cricket_data/Teams/SQUADS.xlsx', 
                        sheet_name='RCB')
team_RCB = team_RCB.sort_values(by='XI').head(11)
team_RCB  = team_RCB.merge(player_list[['Name','Func_Name']], on='Name', how='left')
team_RCB['Func_Name'] = team_RCB['Func_Name'].fillna(team_RCB['Name'])

team_RR = pd.read_excel('/Users/roumyadas/Desktop/Data/t20_cricket_data/Teams/SQUADS.xlsx', 
                        sheet_name='RR')
team_RR = team_RR.sort_values(by='XI').head(11)
team_RR  = team_RR.merge(player_list[['Name','Func_Name']], on='Name', how='left')
team_RR['Func_Name'] = team_RR['Func_Name'].fillna(team_RR['Name'])

team_SRH = pd.read_excel('/Users/roumyadas/Desktop/Data/t20_cricket_data/Teams/SQUADS.xlsx', 
                        sheet_name='SRH')
team_SRH = team_SRH.sort_values(by='XI').head(11)
team_SRH  = team_SRH.merge(player_list[['Name','Func_Name']], on='Name', how='left')
team_SRH['Func_Name'] = team_SRH['Func_Name'].fillna(team_SRH['Name'])

In [71]:
def match_scorecard(match_id):
    ## bowler stats

    df_mod_1 = df_all[(df_all.match_id==match_id)&(df_all.innings==1)]
    df_mod_2 = df_all[(df_all.match_id==match_id)&(df_all.innings==2)]
    
    
    total_1 = df_mod_1['total_runs'].sum()
    total_2 = df_mod_2['total_runs'].sum()
    
    wickets_1 = df_mod_1['isWicket'].sum()
    wickets_2 = df_mod_2['isWicket'].sum()
    
    balls_1 = df_mod_1['legal_balls_bowled'].max()
    balls_2 = df_mod_2['legal_balls_bowled'].max()
    overs_1 = str(balls_1//6)+"."+str(balls_1%6)
    overs_2 = str(balls_2//6)+"."+str(balls_2%6)


    bowler_stats = df_mod_1.groupby(['bowler','bowling_team']).agg(   ##,'innings'
        runs = ('runs_conceeded','sum'),
        balls = ('islegal' ,'sum'),
        wkts = ('isBowlerWicket','sum'),
        dots = ('isDotforbowler','sum')

    ).reset_index()

    bowler_stats['economy'] = 6*bowler_stats['runs']/bowler_stats['balls']

    bowler_stats = bowler_stats.sort_values(['wkts','economy'], ascending=[False, True]).reset_index(drop=True)

    #batting stats

    batter_stats = df_mod_1.groupby(['striker','batting_team']).agg(  ##,'innings'
        runs = ('runs_off_bat','sum'),
        balls = ('is_faced_by_batter' ,'sum'),
        out = ('is_striker_Out','sum'),
        dots = ('isDotforBatter','sum')

    ).reset_index()

    batter_stats['strike_rate'] = 100*batter_stats['runs']/batter_stats['balls']
    for index,row in batter_stats.iterrows():
        batter = row['striker']
        if (batter in df_mod_1.player_dismissed.unique())==True:
            
            new_out_value = 1
            batter_stats.at[index, 'out'] = new_out_value
    
    batter_stats['out'] = batter_stats['out'].replace(0,'not out')
    batter_stats['out'] = batter_stats['out'].replace(1,'out')

    batter_stats = batter_stats.sort_values(['runs','strike_rate'], ascending=[False,False]).reset_index(drop=True)
    
    batter_stats = batter_stats.round(2)
    bowler_stats = bowler_stats.round(2)

    print("innings 1")
    print("BOWLING ---")
    print("^^"*10)
    print(bowler_stats)
    print("BATTING ---")
    print("^^"*10)
    print(batter_stats)
    print("  ")
    print(f"Score :: {total_1}-{wickets_1}  ({overs_1}), run-rate = {np.round(6*total_1/balls_1, 2)}")
    print("***"*10)
    print("***"*10)

    bowler_stats = df_mod_2.groupby(['bowler','bowling_team']).agg(   ##,'innings'
        runs = ('runs_conceeded','sum'),
        balls = ('islegal' ,'sum'),
        wkts = ('isBowlerWicket','sum'),
        dots = ('isDotforbowler','sum')

    ).reset_index()

    bowler_stats['economy'] = 6*bowler_stats['runs']/bowler_stats['balls']

    bowler_stats = bowler_stats.sort_values(['wkts','economy'], ascending=[False, True]).reset_index(drop=True)

    #batting stats

    batter_stats = df_mod_2.groupby(['striker','batting_team']).agg(  ##,'innings'
        runs = ('runs_off_bat','sum'),
        balls = ('is_faced_by_batter' ,'sum'),
        outs = ('is_striker_Out','sum'),
        dots = ('isDotforBatter','sum')

    ).reset_index()

    batter_stats['strike_rate'] = 100*batter_stats['runs']/batter_stats['balls']
    for index,row in batter_stats.iterrows():
        batter = row['striker']
        if batter in df_mod_2.player_dismissed.unique():
            new_out_value = 1
            batter_stats.at[index, 'outs'] = new_out_value
        
    batter_stats['outs'] = batter_stats['outs'].replace(0,'not out')
    batter_stats['outs'] = batter_stats['outs'].replace(1,'out')

    batter_stats = batter_stats.sort_values(['runs','strike_rate'], ascending=[False,False]).reset_index(drop=True)
    
    batter_stats = batter_stats.round(2)
    bowler_stats = bowler_stats.round(2)
    
    
    team1 = df_mod_1.batting_team.unique()[0]
    team2 = df_mod_2.batting_team.unique()[0]
    
    winner = str(team1)+' wins!!' if total_1>total_2 else str(team2)+ ' wins!!' if total_1<total_2\
                                            else "it's a TIE!!!!"

    print("innings 2")
    print("BOWLING ---")
    print("^^"*10)
    print(bowler_stats)
    print("BATTING ---")
    print("^^"*10)
    print(batter_stats)
    print("  ")
    print(f"Score :: {total_2}-{wickets_2}  ({overs_2}), run-rate = {np.round(6*total_2/balls_2, 2)}")
    print("***"*10)
    print("***"*10)
    print(f"{winner}")

In [42]:
import time as time

t1 = time.time()

import random
from datetime import datetime, timedelta

# Define the teams
teams = ["CSK", "DC", "GT","KKR","LSG","MI","PBKS","RCB","RR","SRH"]


In [43]:
matches_list = pd.read_excel('/Users/roumyadas/Desktop/Data/t20_cricket_data/Fixtures/IPL_2024_schedule.xlsx',
                            sheet_name='mod')

In [44]:
df_all = pd.DataFrame()

In [59]:
match_number = 9
with open('run_match_1.py') as f:
    exec(f.read())

match 10:: RCB vs KKR
toss :: RCB wins, and will Bat first!
A Nortje to V Kohli : one
A Nortje to PD Salt : six
A Nortje to PD Salt : one
A Nortje to V Kohli : one
A Nortje to PD Salt : one
A Nortje to V Kohli : one
end of over 1; score :: 11 for 0
---------------
CV Varun to V Kohli : one
CV Varun to PD Salt : two
CV Varun to PD Salt : dot
CV Varun to PD Salt : two
CV Varun to PD Salt : dot
CV Varun to PD Salt : four
end of over 2; score :: 20 for 0
---------------
Harshit Rana to V Kohli : one
Harshit Rana to PD Salt : one
Harshit Rana to V Kohli : dot
Harshit Rana to V Kohli : four
Harshit Rana to V Kohli : legbye
Harshit Rana to PD Salt : six
end of over 3; score :: 33 for 0
---------------
A Nortje to V Kohli : one
A Nortje to PD Salt : wide
A Nortje to PD Salt : wide
A Nortje to PD Salt : two
A Nortje to PD Salt : one
A Nortje to V Kohli : one
A Nortje to PD Salt : one
A Nortje to V Kohli : four
end of over 4; score :: 45 for 0
---------------
VG Arora to PD Salt : wicket
batter 

B Kumar to AD Russell : three, score: 136
B Kumar to Rahmanullah Gurbaz : one, score: 137
B Kumar to AD Russell : one, score: 138
B Kumar to Rahmanullah Gurbaz : three, score: 141
B Kumar to AD Russell : six, score: 147
end of over 12; score :: 147 for 4
---------------
Suyash Sharma to Rahmanullah Gurbaz : one, score: 148
Suyash Sharma to AD Russell : two, score: 150
Suyash Sharma to AD Russell : dot, score: 150
Suyash Sharma to AD Russell : six, score: 156
Suyash Sharma to AD Russell : wicket, score: 156
batter out: AD Russell, for 25
Suyash Sharma to Ramandeep Singh : six, score: 162
end of over 13; score :: 162 for 5
---------------
JR Hazlewood to Rahmanullah Gurbaz : one, score: 163
JR Hazlewood to Ramandeep Singh : four, score: 167
JR Hazlewood to Ramandeep Singh : three, score: 170
JR Hazlewood to Rahmanullah Gurbaz : legbye, score: 171
JR Hazlewood to Rahmanullah Gurbaz : one, score: 172
JR Hazlewood to Ramandeep Singh : three, score: 175
end of over 14; score :: 175 for 5
---

<string>:748: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
<string>:755: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [74]:
print("Match Scorecard")
print("--")

match_scorecard('M010')

Match Scorecard
innings 1
BOWLING ---
^^^^^^^^^^^^^^^^^^^^
         bowler bowling_team  runs  balls  wkts  dots  economy
0     SP Narine          KKR    16     18     2    10     5.33
1      VG Arora          KKR    37     24     2     9     9.25
2  Harshit Rana          KKR    42     24     2     9    10.50
3    AD Russell          KKR    26     12     1     2    13.00
4      CV Varun          KKR    40     18     1     3    13.33
5      A Nortje          KKR    34     24     0     3     8.50
BATTING ---
^^^^^^^^^^^^^^^^^^^^
           striker batting_team  runs  balls      out  dots  strike_rate
0    Suyash Sharma          RCB    41     15      out     1       273.33
1          V Kohli          RCB    33     20      out     4       165.00
2          PD Salt          RCB    27     14      out     5       192.86
3          B Kumar          RCB    26     15      out     5       173.33
4        JM Sharma          RCB    14     11      out     9       127.27
5       JG Bethell          R

In [ ]:
pts_table#.head(5)

In [ ]:
batter_stats.head(15)

In [ ]:
bowler_stats.head(15)